In [2]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
print('# Last run:', datetime.datetime.now().strftime('%Y-%m-%d'))

# Last run: 2024-06-13


# 1. Search lipid traits in GWAS catalog database

In [4]:
fn_gaws_catalog = '/data100t1/home/wanying/shared_data_files/GWAS_catalog/all_associations_v1.0.2.tsv'
df_gwas_catalog = pd.read_csv(fn_gaws_catalog, sep='\t', dtype=str)
print(df_gwas_catalog.shape)
df_gwas_catalog.head()

/tmp/ipykernel_886353/60210075.py:2: DtypeWarning: Columns (11,12,13,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gwas_catalog = pd.read_csv(fn_gaws_catalog, sep='\t')


(621136, 38)


,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
0,2016-01-27,25673413,Locke AE,2015-02-12,Nature,www.ncbi.nlm.nih.gov/pubmed/25673413,Genetic studies of body mass index yield new i...,Body mass index,"up to 104,666 European ancestry male individua...","up to 48,274 European ancestry male individual...",...,6.045757,NaN,0.015,[0.009-0.0208] kg/m2 increase,"Affymetrix, Illumina [2550021]",N,body mass index,http://www.ebi.ac.uk/efo/EFO_0004340,GCST002783,Genome-wide genotyping array
1,2016-01-27,25673413,Locke AE,2015-02-12,Nature,www.ncbi.nlm.nih.gov/pubmed/25673413,Genetic studies of body mass index yield new i...,Body mass index,"up to 104,666 European ancestry male individua...","up to 48,274 European ancestry male individual...",...,6.397940,NaN,0.017,[0.01-0.023] kg/m2 increase,"Affymetrix, Illumina [2550021]",N,body mass index,http://www.ebi.ac.uk/efo/EFO_0004340,GCST002783,Genome-wide genotyping array
2,2016-01-27,25673413,Locke AE,2015-02-12,Nature,www.ncbi.nlm.nih.gov/pubmed/25673413,Genetic studies of body mass index yield new i...,Body mass index,"up to 104,666 European ancestry male individua...","up to 48,274 European ancestry male individual...",...,7.698970,NaN,0.017,[0.011-0.023] kg/m2 increase,"Affymetrix, Illumina [2550021]",N,body mass index,http://www.ebi.ac.uk/efo/EFO_0004340,GCST002783,Genome-wide genotyping array
3,2016-01-27,25673413,Locke AE,2015-02-12,Nature,www.ncbi.nlm.nih.gov/pubmed/25673413,Genetic studies of body mass index yield new i...,Body mass index,"up to 104,666 European ancestry male individua...","up to 48,274 European ancestry male individual...",...,6.154902,NaN,0.017,[0.01-0.023] kg/m2 increase,"Affymetrix, Illumina [2550021]",N,body mass index,http://www.ebi.ac.uk/efo/EFO_0004340,GCST002783,Genome-wide genotyping array
4,2016-01-27,25673413,Locke AE,2015-02-12,Nature,www.ncbi.nlm.nih.gov/pubmed/25673413,Genetic studies of body mass index yield new i...,Body mass index,"up to 104,666 European ancestry male individua...","up to 48,274 European ancestry male individual...",...,6.000000,NaN,0.017,[0.01-0.024] kg/m2 increase,"Affymetrix, Illumina [2550021]",N,body mass index,http://www.ebi.ac.uk/efo/EFO_0004340,GCST002783,Genome-wide genotyping array


In [27]:
cols_to_use = ['MAPPED_TRAIT', 'SNPS', 'SNP_ID_CURRENT', 'CHR_ID', 'CHR_POS', 'P-VALUE', 'OR or BETA',
               'SNP_GENE_IDS', 'REPORTED GENE(S)', 'MAPPED_GENE', 'CONTEXT', 'INTERGENIC']
df_gwas_catalog[cols_to_use].head()

,MAPPED_TRAIT,SNPS,SNP_ID_CURRENT,CHR_ID,CHR_POS,P-VALUE,OR or BETA,SNP_GENE_IDS,REPORTED GENE(S),MAPPED_GENE,CONTEXT,INTERGENIC
0,body mass index,rs968059,968059.0,2,35178945,9.000000e-07,0.015,NaN,LOC100288911,RN7SL602P - SMIM7P1,intron_variant,1.0
1,body mass index,rs972540,972540.0,2,206380059,4.000000e-07,0.017,NaN,ADAM23,ZDBF2 - ACER2P1,intergenic_variant,1.0
2,body mass index,rs977747,977747.0,1,47219005,2.000000e-08,0.017,ENSG00000162367,TAL1,TAL1,3_prime_UTR_variant,0.0
3,body mass index,rs9856151,9856151.0,3,131855641,7.000000e-07,0.017,ENSG00000196353,CPNE4,CPNE4,intron_variant,0.0
4,body mass index,rs9867325,9867325.0,3,136900067,1.000000e-06,0.017,"ENSG00000214283, ENSG00000158092",NCK1,"RAD51AP1P1, NCK1",non_coding_transcript_exon_variant,0.0


In [28]:
len(df_gwas_catalog['MAPPED_TRAIT'].unique())

10493

In [29]:
df_gwas_catalog['MAPPED_TRAIT'].unique()

array(['body mass index', 'brain volume measurement, putamen volume',
       'brain volume measurement, caudate nucleus volume', ...,
       'behavioural ideal cardiovascular health',
       'heptachlor epoxide measurement', 'methoxychlor measurement'],
      dtype=object)

In [56]:
count = 0
keywords_to_include = ['lipid', 'Sphingosine']
# Exclude traits with these keywords
keywords_to_exclude = ['kinase', 'antibody', 'anti', 'hepatic', 'protein', 'lipidemia',
                       'percentage', 'ratio', 'tissue', ' in ']

for val in df_gwas_catalog['DISEASE/TRAIT'].unique():
    flag_include, flag_exclude = False, True
    for v in keywords_to_include:
        flag_include = flag_include or v.upper() in str(val).upper()
    for v in keywords_to_exclude:
        flag_exclude = flag_exclude and v.upper() not in str(val).upper()
    if flag_include and flag_exclude:
        print(val)
        count += 1
print(count)

Lipid traits
Lipid metabolism phenotypes
Sphingolipid levels
Phospholipid levels (plasma)
RBC levels of Sphingosine 1-phosphate
3-ketodihydrosphingosine reductase levels
Sphingosine 1-phosphate levels
Lactosyl-N-palmitoyl-sphingosine (d18:1/16:0) levels
Glycosyl-N-palmitoyl-sphingosine (d18:1/16:0) levels
Lactosyl-N-nervonoyl-sphingosine (d18:1/24:1) levels
Sphingosine levels
N-stearoyl-sphingosine (d18:1/18:0) levels
Glycosyl-N-stearoyl-sphingosine (d18:1/18:0) levels
N-palmitoyl-heptadecasphingosine (d17:1/16:0) levels
Glycosyl-N-(2-hydroxynervonoyl)-sphingosine (d18:1/24:1(2OH)) levels
Disorders of lipid metabolism
Lipid traits (pleiotropy) (HIPO component 2)
Lipid traits (pleiotropy) (HIPO component 1)
Lipid lowering medication use
Glycosyl-N-palmitoyl-sphingosine levels
Lactosyl-N-palmitoyl-sphingosine levels
N-palmitoyl-sphingosine (d18:1/16:0) levels
Palmitoyl-sphingosine-phosphoethanolamine (d18:1/16:0) levels
Lactosyl-N-behenoyl-sphingosine (d18:1/22:0) levels
Hexadecasphingos

In [54]:
count = 0
keywords_to_include = ['lipid', 'Sphingosine']
# Exclude traits with these keywords
keywords_to_exclude = ['kinase', 'antibody', 'anti', 'hepatic', 'protein', 'lipidemia',
                       'percentage', 'ratio', 'tissue', ' in ']

for val in df_gwas_catalog['MAPPED_TRAIT'].unique():
    flag_include, flag_exclude = False, True
    for v in keywords_to_include:
        flag_include = flag_include or v.upper() in str(val).upper()
    for v in keywords_to_exclude:
        flag_exclude = flag_exclude and v.upper() not in str(val).upper()
    if flag_include and flag_exclude:
        print(val)
        count += 1
print(count)

lipid measurement
sphingolipid measurement
glycerophospholipid measurement
phospholipid measurement
sphingosine 1-phosphate measurement
lactosyl-N-palmitoyl-sphingosine (d18:1/16:0) measurement
glycosyl-N-palmitoyl-sphingosine (d18:1/16:0) measurement
lactosyl-N-nervonoyl-sphingosine (d18:1/24:1) measurement
sphingosine measurement
N-stearoyl-sphingosine (d18:1/18:0) measurement
glycosyl-N-stearoyl-sphingosine (d18:1/18:0) measurement
N-palmitoyl-heptadecasphingosine (d17:1/16:0) measurement
glycosyl-N-(2-hydroxynervonoyl)-sphingosine (d18:1/24:1(2OH)) measurement
triglyceride measurement, phospholipid measurement
N-palmitoyl-sphingosine (d18:1/16:0) measurement
palmitoyl-sphingosine-phosphoethanolamine (d18:1/16:0) measurement
lactosyl-N-behenoyl-sphingosine (d18:1/22:0) measurement
hexadecasphingosine (d16:1) measurement
18


In [55]:
df_gwas_catalog[df_gwas_catalog['MAPPED_TRAIT'] == 'lipid measurement']

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
53740,2020-06-17,32180562,Kulyte A,2020-01-25,Mol Metab,www.ncbi.nlm.nih.gov/pubmed/32180562,Genome-wide association study of adipocyte lip...,Spontaneous adipocyte lipolysis,856 European ancestry individuals,NaN,...,6.301030,NaN,0.144000,[0.087-0.201] unit increase,NR [9260588] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST010088,Genome-wide genotyping array
53741,2020-06-17,32180562,Kulyte A,2020-01-25,Mol Metab,www.ncbi.nlm.nih.gov/pubmed/32180562,Genome-wide association study of adipocyte lip...,Spontaneous adipocyte lipolysis,856 European ancestry individuals,NaN,...,5.522879,NaN,0.449000,[0.26-0.64] unit increase,NR [9260588] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST010088,Genome-wide genotyping array
53742,2020-06-17,32180562,Kulyte A,2020-01-25,Mol Metab,www.ncbi.nlm.nih.gov/pubmed/32180562,Genome-wide association study of adipocyte lip...,Spontaneous adipocyte lipolysis,856 European ancestry individuals,NaN,...,5.522879,NaN,0.429000,[0.25-0.61] unit increase,NR [9260588] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST010088,Genome-wide genotyping array
53743,2020-06-17,32180562,Kulyte A,2020-01-25,Mol Metab,www.ncbi.nlm.nih.gov/pubmed/32180562,Genome-wide association study of adipocyte lip...,Spontaneous adipocyte lipolysis,856 European ancestry individuals,NaN,...,5.221849,NaN,0.095000,[0.054-0.136] unit increase,NR [9260588] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST010088,Genome-wide genotyping array
53744,2020-06-17,32180562,Kulyte A,2020-01-25,Mol Metab,www.ncbi.nlm.nih.gov/pubmed/32180562,Genome-wide association study of adipocyte lip...,Spontaneous adipocyte lipolysis,856 European ancestry individuals,NaN,...,5.698970,NaN,0.097000,[0.058-0.136] unit increase,NR [9260588] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST010088,Genome-wide genotyping array
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574188,2022-04-25,35393526,Montasser ME,2022-04-07,Commun Biol,www.ncbi.nlm.nih.gov/pubmed/35393526,An Amish founder population reveals rare-popul...,Glycosphingolipids (d42:1) levels,650 Amish individuals,847 European American ancestry individuals,...,8.154902,(Amish),0.948985,unit decrease,Affymetrix [7917458] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST90101997,Genome-wide genotyping array
574189,2022-04-25,35393526,Montasser ME,2022-04-07,Commun Biol,www.ncbi.nlm.nih.gov/pubmed/35393526,An Amish founder population reveals rare-popul...,Glycosphingolipids (d42:1) levels,650 Amish individuals,847 European American ancestry individuals,...,7.522879,(Amish),0.881780,unit decrease,Affymetrix [7917458] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST90101997,Genome-wide genotyping array
574190,2022-04-25,35393526,Montasser ME,2022-04-07,Commun Biol,www.ncbi.nlm.nih.gov/pubmed/35393526,An Amish founder population reveals rare-popul...,Glycosphingolipids (d42:1) levels,650 Amish individuals,847 European American ancestry individuals,...,8.096910,(Amish),0.930039,unit decrease,Affymetrix [7917458] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST90101997,Genome-wide genotyping array
574191,2022-04-25,35393526,Montasser ME,2022-04-07,Commun Biol,www.ncbi.nlm.nih.gov/pubmed/35393526,An Amish founder population reveals rare-popul...,Glycosphingolipids (d42:1) levels,650 Amish individuals,847 European American ancestry individuals,...,7.698970,(Amish),0.906767,unit decrease,Affymetrix [7917458] (imputed),N,lipid measurement,http://www.ebi.ac.uk/efo/EFO_0004529,GCST90101997,Genome-wide genotyping array


# 2. Combine GWAS results

# 3. Create manhattan plot with known hits